# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
S. v. Terwisga  ->  S. v. Terwisga  |  ['S. v. Terwisga']
Arxiv has 41 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2402.14135


extracting tarball to tmp_2402.14135... done.
Retrieving document from  https://arxiv.org/e-print/2402.14535


extracting tarball to tmp_2402.14535...

 done.


...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2402.14653/ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'combined_line_plot_function' from 'tmp_2402.14653/combined_line_plot_function.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'combined_line_plot_function' from 'tmp_2402.14653/combined_line_plot_function.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/host

/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure Figures/CO_images/composite/#1/ANALYSIS/#2/#1-#2-composite.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:692: LatexWarning: Could not find graphic \includegraphics[width=0.72\hsize,clip]{Figures/CO_images/composite/#1/ANALYSIS/#2/#1-#2-composite.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure Figures/CO_images/kepler/#1/ANALYSIS/#2/#1-#2-kepler.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:692: LatexWarning: Could not find graphic \includegraphics[width=0.

/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure Figures/CO_images/composite/#1/ANALYSIS/#2/#1-#2-composite.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:692: LatexWarning: Could not find graphic \includegraphics[width=0.72\hsize,clip]{Figures/CO_images/composite/#1/ANALYSIS/#2/#1-#2-composite.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure Figures/CO_images/kepler/#1/ANALYSIS/#2/#1-#2-kepler.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:692: LatexWarning: Could not find graphic \includegraphics[width=0.

Found 159 bibliographic references in tmp_2402.14653/ms.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.14535-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.14535) | **Quenching-driven equatorial depletion and limb asymmetries in hot  Jupiter atmospheres: WASP-96b example**  |
|| M. Zamyatina, et al. -- incl., <mark>E.-M. Ahrer</mark> |
|*Appeared on*| *2024-02-23*|
|*Comments*| *accepted to MNRAS, 27 pages, 24 figures*|
|**Abstract**| Transport-induced quenching in hot Jupiter atmospheres is a process that determines the boundary between the part of the atmosphere at chemical equilibrium and the part of the atmosphere at thermochemical (but not photothermochemical) disequilibrium. The location of this boundary, the quench level, depends on the interplay between the dynamical and chemical timescales in the atmosphere, with quenching occurring when these timescales are equal. We explore the sensitivity of the quench level position to an increase in the planet's atmospheric metallicity using aerosol-free 3D GCM simulations of a hot Jupiter WASP-96b. We find that the temperature increase at pressures of $\sim$$10^{4}-10^{7}$ Pa that occurs when metallicity is increased could shift the position of the quench level to pressures dominated by the jet, and cause an equatorial depletion of $CH_4$, $NH_3$ and $HCN$. We discuss how such a depletion affects the planet's transmission spectrum, and how the analysis of the evening-morning limb asymmetries, especially within $\sim3-5 {\mu}m$, could help distinguish atmospheres of different metallicities that are at chemical equilibrium from those with the upper layers at thermochemical disequilibrium. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.14653-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.14653) | **PRODIGE -- Planet-forming disks in Taurus with NOEMA. I. Overview and  first results for 12CO, 13CO, and C18O**  |
|| <mark>D. Semenov</mark>, et al. -- incl., <mark>K. Schwarz</mark>, <mark>S. v. Terwisga</mark> |
|*Appeared on*| *2024-02-23*|
|*Comments*| *47 pages, 10 figures, accepted for publication by A&A*|
|**Abstract**| We are performing a line survey of 8 planet-forming Class II disks in Taurus with the IRAM NOrthern Extended Millimeter Array (NOEMA), as a part of the MPG-IRAM Observatory Program PRODIGE (PROtostars and DIsks: Global Evolution; PIs: P. Caselli and Th. Henning). Compact and extended disks around T Tauri stars CI, CY, DG, DL, DM, DN, IQ Tau, and UZ Tau E are observed in ~80 lines from >20 C-, O,- N-, and S-bearing species. The observations in four spectral settings at 210-280 GHz with $1\sigma$ rms sensitivity of ~8-12 mJy/beam at 0.9" and 0.3 km/s resolution will be completed in 2024. The uv-visibilities are fitted with the DiskFit model to obtain key stellar and disk properties. In this paper, the combined $^{12}$CO, $^{13}$CO and C$^{18}$O $J = 2-1$ data are presented. We find that the CO fluxes and disk masses inferred from dust continuum tentatively correlate with the CO emission sizes. We constrain dynamical stellar masses, geometries, temperatures, the CO column densities and gas masses for each disk. The best-fit temperatures at 100 au are ~17-37 K, and decrease radially with the power-law exponent q ~ 0.05-0.76. The inferred CO column densities decrease radially with the power-law exponent p ~ 0.2-3.1. The gas masses estimated from $^{13}$CO (2-1) are ~ $0.001-0.2 M_\textrm{Sun}$. The best-fit CO column densities point to severe CO freeze-out in the disks. The DL Tau disk is an outlier, and has either stronger CO depletion or lower gas mass than the rest of the sample. The CO isotopologue ratios are roughly consistent with the observed values in disks and the low-mass star-forming regions. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.14135-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.14135) | **Steep-spectrum AGN in eROSITA Final Equatorial-Depth Survey (eFEDS):  Their host galaxies and multi-wavelength properties**  |
|| K. Iwasawa, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-02-23*|
|*Comments*| *8 pages, accepted for publication in A&A*|
|**Abstract**| We selected sources with a steep soft-X-ray-band spectrum with a photon index larger than 2.5 -- measured by eROSITA on board the Spectrum-Roentgen-Gamma (SRG) -- from the eFEDS AGN catalogue as candidates of highly accreting supermassive black holes, and investigated their multi-wavelength properties. Among 601 bright AGN with 0.2-5 keV counts of greater than 100, 83 sources (~14%) are classified as steep-spectrum sources. These sources have typical 0.5-2 keV luminosities of L(SX) ~ 1e44 erg/s and the majority of them are found at redshifts below z=1. In comparison with sources with flatter spectra, these sources have, on average, a UV (or optical) to 2 keV luminosity ratio that is larger by ~0.3 dex and bluer optical-to-UV continuum emission. They also appear to be radio quiet based on the detection rate in the FIRST and VLASS surveys. Their host galaxies -- at least in the redshift range of z=0.2-0.8, where the AGN-galaxy decomposition results from the Subaru Hyper Suprime-Cam imaging are available -- tend to be late-type and have smaller stellar masses than those of sources with flatter spectra. These properties are similar to those found in nearby narrow-line Seyfert 1 galaxies, in agreement with the picture that they are AGN with elevated accretion rates and are in the early growth phase of black hole and galaxy co-evolution. However, the steep-spectrum sources are not exclusively narrow-line Seyfert 1 galaxies; indeed many are broad-line Seyfert 1 galaxies, as found by a catalogue search. This suggests that these steep-spectrum sources may be black holes generally with high accretion rates but of a wide mass range, including a few objects emitting at L(SX)>1e45 erg/s, of which black hole masses can be close to 10^9 M_sun. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2402.14535.md
    + _build/html/tmp_2402.14535/./figures/transmission/wasp96b_transpec_wl02_30_combores_full_mor_eve.png
    + _build/html/tmp_2402.14535/./figures/chemistry/wasp96b_midtransit_ch4_w_ch4_qplevs_w_diff.png
    + _build/html/tmp_2402.14535/./figures/circulation/wasp96b_vp_pres_temp_limbs.png
exported in  _build/html/2402.14653.md
    + _build/html/tmp_2402.14653/./Figures/CO_images/kepler/DM_Tau/ANALYSIS/CO/DM_Tau-CO-kepler.png
    + _build/html/tmp_2402.14653/./Figures/CO_images/kepler/DM_Tau/ANALYSIS/CO/DM_Tau-CO-kepler.png
    + _build/html/tmp_2402.14653/./Figures/CO_images/composite/CI_Tau/ANALYSIS/CO/CI_Tau-CO-composite.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Quenching-driven equatorial depletion and limb asymmetries in hot Jupiter atmospheres: WASP-96b example

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2402.14535-b31b1b.svg)](https://arxiv.org/abs/2402.14535)<mark>Appeared on: 2024-02-23</mark> -  _accepted to MNRAS, 27 pages, 24 figures_

</div>
<div id="authors">

M. Zamyatina, et al. -- incl., <mark>E.-M. Ahrer</mark>

</div>
<div id="abstract">

**Abstract:** Transport-induced quenching in hot Jupiter atmospheres is a process that determines the boundary between the part of the atmosphere at chemical equilibrium and the part of the atmosphere at thermochemical (but not photothermochemical) disequilibrium. The location of this boundary, the quench level, depends on the interplay between the dynamical and chemical timescales in the atmosphere, with quenching occurring when these timescales are equal. We explore the sensitivity of the quench level position to an increase in the planet's atmospheric metallicity using aerosol-free 3D GCM simulations of a hot Jupiter WASP-96b. We find that the temperature increase at pressures of $\sim$ $\SIrange{e4}{e7}{\pascal}$ that occurs when metallicity is increased could shift the position of the quench level to pressures dominated by the jet, and cause an equatorial depletion of $\ce{CH4}$ , $\ce{NH3}$ and $\ce{HCN}$ . We discuss how such a depletion affects the planet's transmission spectrum, and how the analysis of the evening-morning limb asymmetries, especially within $\sim$ $\SIrange{3}{5}{\micro\metre}$ , could help distinguish atmospheres of different metallicities that are at chemical equilibrium from those with the upper layers at thermochemical disequilibrium.

</div>

<div id="div_fig1">

<img src="tmp_2402.14535/./figures/transmission/wasp96b_transpec_wl02_30_combores_full_mor_eve.png" alt="Fig6" width="100%"/>

**Figure 6. -** WASP-96b transmission spectra predicted by the Met Office Unified Model equilibrium and kinetics simulations assuming $[M/H]=0$ or $[M/H]=1$. Panel (a) shows the full limb (morning plus evening) spectra, and (b) the morning and evening limb spectra. Panel (c) shows the difference between the full limb spectra due to the disequilibrium thermochemistry, and panel (d) shows the difference between evening and morning limb spectra (evening minus morning). Faint grey dotted lines indicate the location of zero to better guide the eye. (*fig:wasp96b_transpec_wl02_30_combores_full_mor_eve*)

</div>
<div id="div_fig2">

<img src="tmp_2402.14535/./figures/chemistry/wasp96b_midtransit_ch4_w_ch4_qplevs_w_diff.png" alt="Fig4" width="100%"/>

**Figure 4. -** Pressure-\ce{CH4} cross-sections through WASP-96b's limb ("mid-transit view") according to the equilibrium and kinetics simulations assuming $[M/H]=0$ or $[M/H]=1$. Panels (a-d) show \ce{CH4} distribution in each simulation. Panels (e-f) show the difference caused by an increase in metallicity, and (g-h) the difference caused by disequilibrium thermochemistry. Panels (a-d) and (e-h) share the top and bottom colour scale in the bottom right of the figure, respectively. The black thick lines in panels (b) and (d) indicate the location of \ce{CH4} quench level. All panels show variable distributions at longitudes \ang{90}E and \ang{270}E exactly (not the average over the opening angle). (*fig:wasp96b_pres_ch4_in_transit*)

</div>
<div id="div_fig3">

<img src="tmp_2402.14535/./figures/circulation/wasp96b_vp_pres_temp_limbs.png" alt="Fig11" width="100%"/>

**Figure 11. -** Pressure-temperature vertical profiles at the morning and evening limb from the WASP-96b equilibrium and kinetics simulations assuming $[M/H]=0$ or $[M/H]=1$. (*fig:wasp96b_vp_pres_temp_limbs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2402.14535"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\setvalue}[1]{\pgfkeys{/variables/#1}}$
$\newcommand{\getvalue}[1]{\pgfkeysvalueof{/variables/#1}}$
$\newcommand{\declare}[1]{$
$ \pgfkeys{$
$  /variables/#1.is family,$
$  /variables/#1.unknown/.style = {\pgfkeyscurrentpath/\pgfkeyscurrentname/.initial = ##1}$
$ }$
$}$
$\newcommand$
$\newcommand$
$\newcommand{\textsg}[1]{\textcolor{black}{#1}}$
$\newcommand{\textds}[1]{\textcolor{black}{#1}}$
$\newcommand{\upd}[1]{\textcolor{black}{#1}}$
$\newcommand{\sgupd}[1]{\textcolor{black}{#1}}$
$\newcommand{\setvalue}[1]{\pgfkeys{/variables/#1}}$
$\newcommand{\getvalue}[1]{\pgfkeysvalueof{/variables/#1}}$
$\newcommand{\declare}[1]{$
$ \pgfkeys{$
$  /variables/#1.is family,$
$  /variables/#1.unknown/.style = {\pgfkeyscurrentpath/\pgfkeyscurrentname/.initial = ##1}$
$ }$
$}$
$\newcommand$
$\newcommand$
$\newcommand{\textsg}[1]{\textcolor{black}{#1}}$
$\newcommand{\textds}[1]{\textcolor{black}{#1}}$
$\newcommand{\upd}[1]{\textcolor{black}{#1}}$
$\newcommand{\sgupd}[1]{\textcolor{black}{#1}}$</div>



<div id="title">

# PRODIGE - Planet-forming disks in Taurus with NOEMA.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2402.14653-b31b1b.svg)](https://arxiv.org/abs/2402.14653)<mark>Appeared on: 2024-02-23</mark> -  _47 pages, 10 figures, accepted for publication by A&A_

</div>
<div id="authors">

<mark>D. Semenov</mark>, et al. -- incl., <mark>K. Schwarz</mark>, <mark>S. v. Terwisga</mark>

</div>
<div id="abstract">

**Abstract:** Physics and chemistry of planet-forming disks are far from being fully understood. $\upd{To make further progress, both broad line surveys and observations of individual tracers in}$ a statistically-significant number of disks are required. We are performing a line survey of $\upd{8 planet-forming Class II disks in Taurus}$ with the IRAM NOrthern Extended Millimeter Array (NOEMA), as a part of the MPG-IRAM Observatory Program PRODIGE (PROtostars and DIsks: Global Evolution; PIs: P. Caselli and Th. Henning). Compact and extended disks around T Tauri stars CI, CY, DG, DL, DM, DN, IQ Tau, and UZ Tau E are observed $\upd{in $\sim 80$ lines from $>20$ C-, O,- N-, and S-bearing species.}$ The observations $\upd{in four spectral settings at 210-280 GHz with $1\sigma$ rms sensitivity}$ of $\sim 8-12$ mJy/beam at $0.9\arcsec$ and 0.3 km s $^{-1}$ resolution $\upd{will be completed in 2024.The $uv$-visibilities}$ are fitted with the DiskFit model to obtain key stellar and disk properties. $\upd{In this first paper, the combined $^{12}$CO, $^{13}$CO and C$^{18}$O $J=2-1$ data are presented. We find that the CO fluxes and disk masses inferred from dust continuum tentatively correlate with the CO emission sizes. We constrain dynamical stellar masses, geometries, temperatures, the CO column densities and gas masses for each disk.}$ The best-fit temperatures $\upd{at 100 au}$ are $\sim 17-37$ K, $\upd{and decrease radially with the power-law exponent $q \sim 0.05-0.76$. The inferred CO column densities decrease radially with the power-law exponent $p \sim 0.2-3.1$. The gas masses estimated from $^{13}$CO (2-1) are $\sim 0.001 - 0.2 M_\odot$.}$ $\upd{Using NOEMA, we confirm the presence of temperature gradients in our disk sample. The best-fit CO column densities point to severe CO freeze-out in these disks. The DL Tau disk is an outlier, and has either stronger CO depletion or lower gas mass than the rest of the sample. The CO isotopologue ratios are roughly consistent with the observed values in disks and the low-mass star-forming regions. The high $^{13}$CO/C$^{18}$O ratio of $\sim 23$ in DM Tau could be indicative of strong selective photodissociation of C$^{18}$O in this disk. }$

</div>

<div id="div_fig1">

<img src="tmp_2402.14653/./Figures/CO_images/kepler/DM_Tau/ANALYSIS/CO/DM_Tau-CO-kepler.png" alt="Fig16" width="100%"/>

**Figure 16. -** Observations of $^{12}$CO (2-1) emission in the DM Tau disk. Shown is a pixel-deprojected Keplerian plot consisting of the three panels: (left) the radial profile of the line brightness temperature (K) $\upd${with observations and errorbars in black, and profile derived from the best-fit disk model in red. The magenta dashed curve indicates the expected brightness temperature of an optically thick line derived from temperature power law}. (Top) the integrated spectrum (black line) overlaid with the best-fit Gaussian profile (red line). (Bottom right) aligned and stacked line intensity (K) as a function of disk radius (Y-axis; au) and velocity (X-axis; km s$^{-1}$). The color bar units are in Kelvin. (*fig:DM_Tau-CO_kepl*)

</div>
<div id="div_fig2">

<img src="tmp_2402.14653/./Figures/CO_images/kepler/DM_Tau/ANALYSIS/CO/DM_Tau-CO-kepler.png" alt="Fig22" width="100%"/>

**Figure 22. -** Observations of $^{12}$CO (2-1) emission in the DM Tau disk. Shown is a pixel-deprojected Keplerian plot consisting of the three panels: (left) the radial profile of the line brightness temperature (K) $\upd${with observations and errorbars in black, and profile derived from the best-fit disk model in red. The magenta dashed curve indicates the expected brightness temperature of an optically thick line derived from temperature power law}. (Top) the integrated spectrum (black line) overlaid with the best-fit Gaussian profile (red line). (Bottom right) aligned and stacked line intensity (K) as a function of disk radius (Y-axis; au) and velocity (X-axis; km s$^{-1}$). The color bar units are in Kelvin. (*fig:DM_Tau-CO_kepl*)

</div>
<div id="div_fig3">

<img src="tmp_2402.14653/./Figures/CO_images/composite/CI_Tau/ANALYSIS/CO/CI_Tau-CO-composite.png" alt="Fig26" width="100%"/>

**Figure 26. -** The observed $^{12}$CO (2-1) emission in the CI Tau disk.
Shown are the channel map of the observed line brightness distribution (top left), the moment-zero map (top right), and the integrated spectrum (top middle). The channel map shows 16 velocity channels with a step of $0.5$ km s$^{-1}$ in the [-4.5, +4.5] km s$^{-1}$ range around the systemic velocity. The contour lines start at $2$ K, with a step of $2$ K. The color bar shows line brightness temperatures (K). The contour lines in the moment-zero plot start at 3, 6, and $9\sigma$, with a step of $6\sigma$ afterward. The $1\sigma$ rms noise (mJy km s$^{-1}$) is shown in the upper left corner of the moment-zero plot, and the synthesized beam is depicted by the dark ellipse in the left bottom corner. The $^{12}$CO (2-1) spectrum shows a nearly absent blueshifted velocity component due to the cloud absorption. (*fig:CI_Tau-CO_comp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2402.14653"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

374  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
